In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.notebook import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ptbdata/WFDB/S0204.mat
/kaggle/input/ptbdata/WFDB/S0272.mat
/kaggle/input/ptbdata/WFDB/S0538.hea
/kaggle/input/ptbdata/WFDB/S0503.mat
/kaggle/input/ptbdata/WFDB/S0548.hea
/kaggle/input/ptbdata/WFDB/S0364.hea
/kaggle/input/ptbdata/WFDB/S0340.mat
/kaggle/input/ptbdata/WFDB/S0192.hea
/kaggle/input/ptbdata/WFDB/S0312.mat
/kaggle/input/ptbdata/WFDB/S0490.mat
/kaggle/input/ptbdata/WFDB/S0518.mat
/kaggle/input/ptbdata/WFDB/S0235.hea
/kaggle/input/ptbdata/WFDB/S0079.hea
/kaggle/input/ptbdata/WFDB/S0410.mat
/kaggle/input/ptbdata/WFDB/S0306.mat
/kaggle/input/ptbdata/WFDB/S0472.mat
/kaggle/input/ptbdata/WFDB/S0135.mat
/kaggle/input/ptbdata/WFDB/S0201.mat
/kaggle/input/ptbdata/WFDB/S0437.hea
/kaggle/input/ptbdata/WFDB/S0042.hea
/kaggle/input/ptbdata/WFDB/S0281.hea
/kaggle/input/ptbdata/WFDB/S0028.mat
/kaggle/input/ptbdata/WFDB/S0463.mat
/kaggle/input/ptbdata/WFDB/S0079.mat
/kaggle/input/ptbdata/WFDB/S0251.mat
/kaggle/input/ptbdata/WFDB/S0515.hea
/kaggle/input/ptbdata/WFDB/S0243.mat
/

In [2]:
#schema: id, name, segment, data, label, features
import wfdb
import base64
import datetime
import gzip
label_count = {}

def downsample(nums):
    nums = [nums[i] for i in range(len(nums)) if i%2==0]
    return np.array(nums)

def prepare_data(dir_add, outfile):
    
    ids = os.listdir(dir_add)
    ids = [item for item in ids if item.find('patient') != -1]
    ids = sorted(ids)
    
    output_str = []
    belongs = []
    fre_dict = {}
    
    for id in tqdm(ids):
        names = os.listdir(dir_add+id)
        names = [name[:-4] for name in names]
        names = list(set(names))

        for name in names:
            file_add = dir_add + id + "/" + name
            datas = wfdb.rdsamp(file_add)
            data = downsample(datas[0][:,:12]).transpose(1,0)
            
            label = 0
            assert data.shape[0] == 12
            num_sample = int(data.shape[1]/5000)
            for k in range(num_sample):
                #print(5000*k, 5000*(k+1))
                sub_data = data[:,5000*k:5000*(k+1)]
                output_str.append([id, name, str(k+fre_dict.get(id,0)), base64.b64encode(np.ascontiguousarray(sub_data)).decode("utf-8"), base64.b64encode(np.array([0])).decode("utf-8"), '^1'.join(datas[1]['comments'])])
                belongs.append(id)
            fre_dict[id] = fre_dict.get(id,0) + num_sample
    return output_str, belongs, fre_dict  #5724

output_str, belongs, fre_dict = prepare_data('../input/ptbdatas/ptb-diagnostic-ecg-database-1.0.0/', 'ptb_reg_datas')

In [3]:
train_set = []  #5181
test_set = []  #543
num_tests = []
for item in tqdm(output_str):
    id,name,k,data,label,features = item
    k_all = fre_dict[id]
    num_test = int(k_all/10)
    if num_test == 0: num_test = 1
    k_new = k+"/"+str(k_all)
    output = [id, name, k_new, data, label, features]
    if k == '0' : num_tests.append(num_test)
    if int(k) < num_test: 
        test_set.append(output)
    else: train_set.append(output)

In [4]:
import random
random.shuffle(train_set)

In [5]:
for i, line in enumerate(train_set):
    with open('ptb_train'+".part"+str(i).zfill(4),'w+') as f:
        f.write('\t'.join(line) + '\n')
for i, line in enumerate(test_set):
    with open('ptb_test'+".part"+str(i).zfill(4),'w+') as f:
        f.write('\t'.join(line) + '\n')